In [2]:
# 要引入的包

import numpy as np
from matplotlib import pyplot as plt
import cv2
from imutils.perspective import four_point_transform
from PIL import Image, ImageDraw, ImageFont

# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'

In [3]:
# 定义一坨函数

def cvImread(file_path):
    cv_img = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), -1)
    return cv_img
    
def cvImsave(img, file_path, img_type):
    cv2.imencode(img_type, img)[1].tofile(file_path)

def showImage(img):
    cv2.namedWindow("Test", cv2.WINDOW_NORMAL)
    cv2.imshow("Test", img)
    cv2.waitKey()
    k = cv2.waitKey()
    if k == 27:
        cv2.destroyAllWindows()

def binaryPreprocess(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGBA2GRAY)
    blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    ret, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th

def cutEdge(img_binary):
    top = 0
    bottom = -1
    left = 0
    right = -1
    while True:
        if np.mean(img_binary[top, :]) < 128:
            top += 1
        else:
            break
    while True:
        if np.mean(img_binary[bottom, :]) < 128:
            bottom -= 1
        else:
            break
    while True:
        if np.mean(img_binary[:, left]) < 128:
            left += 1
        else:
            break
    while True:
        if np.mean(img_binary[:, right]) < 128:
            right -= 1
        else:
            break
    
    img_binary = img_binary[top:, left:]
    if bottom != -1:
        img_binary = img_binary[:bottom + 1, :]
    if right != -1:
        img_binary = img_binary[:, :right + 1]

    row, column = np.shape(img_binary)
    white_width_row = np.int(row * 0.01)
    white_width_column = np.int(column * 0.01)
    img_binary[:white_width_row, :] = 255
    img_binary[-white_width_row:, :] = 255
    img_binary[:, :white_width_column] = 255
    img_binary[:, -white_width_column:] = 255
    
    return img_binary

def paint_chinese_opencv(im, chinese, pos, color):
    img_PIL = Image.fromarray(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    myfont = ImageFont.truetype('‪C:\\Windows\\Fonts\\msyhbd.ttc', 96)
    fill_color = color
    position = pos
    if not isinstance(chinese, str):
        chinese = chinese.decode('utf-8')
        draw = ImageDraw.Draw(img_PIL)
        draw.text(position, chinese, font=myfont, fill=fill_color)
        im = cv2.cvtColor(np.asarray(img_PIL), cv2.COLOR_RGB2BGR)
    return im

def compLeafArea(img_file_list, area_of_the_background, num_of_leaves):
    for img_file in img_file_list:
        img = cvImread('./胡海瑶的叶子们/' + img_file + '.jpg')
        img_binary = binaryPreprocess(img)
        img_cut = cutEdge(img_binary)
        contours, hier = cv2.findContours(img_cut, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        contours.sort(key = cv2.contourArea, reverse=True)
        for i in range(1, 1 + num_of_leaves):
            c = contours[i]
            area_prop = cv2.contourArea(c)/np.size(img_cut)
            rect = cv2.minAreaRect(c) # 计算出包围目标的最小区域
            box = cv2.boxPoints(rect) # 计算最小面积矩形的坐标
            box = np.int0(box) # 坐标归一化为整型
            leaf_position = np.int0((box[0] + box[2])/2)
            cv2.putText(img, str(np.round(area_prop * area_of_the_background, 4)), tuple(leaf_position), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (161, 0, 116), 5)
        # showImage(img)
        cvImsave(img, './area/' + img_file + '.jpg', '.jpg')

In [70]:
# 运行这里开始识别

img_file_list = ['桂树1', '桂树2', '桂树3', '桂树4', '桂树5',\
    '银杏1', '银杏2', '银杏3', '银杏4', '银杏5',\
        '紫薇1', '紫薇2', '紫薇3', '紫薇4', '紫薇5']
compLeafArea(img_file_list, 400, 4)

img_file_list = ['垂柳1', '垂柳2', '垂柳3', '垂柳4']
compLeafArea(img_file_list, 400, 1)